## MVTEc  LOCO  Visualizer

- claculates clip embeddings
- vizualisation via tensorboard projector

from: "https://www.mydrive.ch/shares/48237/1b9106ccdfbb09a0c414bd49fe44a14a/download/430647091-1646842701/mvtec_loco_anomaly_detection.tar.xz"

In [1]:
# activate clip env
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import os
import clip
import torch
import pandas as pd

import warnings
warnings.filterwarnings("ignore", message=".*pandas only supports SQLAlchemy connectable.*")


torch.cuda.empty_cache()
print(f'GPU is available: {torch.cuda.is_available()}')
print("Torch version:", torch.__version__)


from PIL import Image

import torchvision
import torch.utils.tensorboard 
from torch.utils.tensorboard import SummaryWriter


GPU is available: True
Torch version: 2.0.1+cu117


2023-10-20 12:55:19.065433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-20 12:55:19.065468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-20 12:55:19.065512: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 12:55:19.078173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 12:55:20.509849: W tensorflow/compiler/

In [6]:

def convert_images_for_tensorboard(image_paths):
    # Preprocess the images
    transform_display = torchvision.transforms.Compose([
        torchvision.transforms.Resize(64),  # Resize the images 
        torchvision.transforms.CenterCrop(64),  # Crop the images
        torchvision.transforms.ToTensor()  # Convert images to tensors        
    ])

    images = []  # Array to store the preprocessed images
    for path in image_paths:
        image = Image.open(path).convert('RGB')  # Open the image and convert to RGB
        # Apply the display transformation
        image_display = transform_display(image)
        image_display = image_display.unsqueeze(0) # Add a batch dimension to the image
        images.append(image_display)

    images = torch.cat(images, dim=0)  # Concatenate the images along the batch dimension

    return images

In [2]:
# PATHs
PROJECT_DATA_PATH='/home/bule/projects/Transformaly_FUAD/data/mvtec_loco'
LOG_DIR= '/home/bule/projects/Transformaly_FUAD/tensorboard_logs'

In [3]:
# paths to images
image_paths = []
for root, dirs, files in os.walk(PROJECT_DATA_PATH):
    if 'ground_truth' not in root:
        for file in files:
            if file.endswith('.png'):
                image_paths.append(os.path.join(root, file))
len(image_paths)
## TODO embedd all images from MVTEC in CLIP space


3651

In [4]:
BATCH_SIZE = 1024  # Adjust this based on your GPU memory
FILENAME = 'MVTEC_LOCO_embeddings_df'

if not os.path.exists(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl')):

    # Load the model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, transform = clip.load('ViT-B/32', device=device)


    # Function to get embeddings for a batch of images
    def get_batch_embeddings(batch_paths):
        images = [Image.open(p) for p in batch_paths]
        tensors = [transform(img) for img in images]
        batch_tensor = torch.stack(tensors).to(device)
        with torch.no_grad():
            batch_features = model.encode_image(batch_tensor)
        return batch_features.cpu().numpy()

    # Calculate embeddings for all images in batches
    all_embeddings = []
    for i in range(0, len(image_paths), BATCH_SIZE):
        batch_paths = image_paths[i:i+BATCH_SIZE]
        batch_embeddings = get_batch_embeddings(batch_paths)
        all_embeddings.extend(batch_embeddings)
        print(f"Processed images {i} to {i + len(batch_embeddings)}")

    # Convert all embeddings to DataFrame and save
    df = pd.DataFrame(all_embeddings, index=image_paths)
    df.to_pickle(os.path.join(PROJECT_DATA_PATH, FILENAME + '.pkl'))
else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

df.head()


Processed images 0 to 1024
Processed images 1024 to 2048
Processed images 2048 to 3072
Processed images 3072 to 3651


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
/home/bule/projects/Transformaly_FUAD/data/mvtec_loco/pushpins/train/good/058.png,0.291504,-0.257324,-0.166260,0.166992,0.115601,-0.413818,0.155640,0.677246,0.120605,-0.269775,...,0.238281,-0.167847,0.216797,-0.140625,-0.104187,-0.401855,0.012543,0.634766,0.242188,-0.396484
/home/bule/projects/Transformaly_FUAD/data/mvtec_loco/pushpins/train/good/341.png,0.059814,-0.027466,-0.314697,0.156494,0.281006,-0.318359,0.069519,0.854004,-0.032806,-0.200806,...,0.214111,-0.184326,0.374268,-0.111023,-0.005463,-0.472656,0.038635,0.823242,-0.063965,-0.292480
/home/bule/projects/Transformaly_FUAD/data/mvtec_loco/pushpins/train/good/065.png,0.102234,-0.143066,-0.223145,0.249756,0.186279,-0.370117,0.138794,0.845703,0.255371,-0.321045,...,0.314453,-0.099060,0.457275,-0.095764,-0.091003,-0.406738,0.200562,0.809570,0.085693,-0.515625
/home/bule/projects/Transformaly_FUAD/data/mvtec_loco/pushpins/train/good/276.png,0.273682,-0.185791,-0.247803,0.176147,0.162964,-0.245483,0.161133,0.826172,0.281982,-0.256104,...,0.288818,-0.256104,0.321289,-0.109314,-0.142212,-0.311279,0.100769,0.744629,0.093323,-0.436279
/home/bule/projects/Transformaly_FUAD/data/mvtec_loco/pushpins/train/good/173.png,-0.015854,-0.005173,-0.145508,0.160034,0.241455,-0.332520,0.112915,0.846680,0.108276,-0.311768,...,0.313965,-0.192139,0.338379,-0.103882,-0.021957,-0.434326,0.043304,0.850098,-0.007778,-0.401123


In [7]:
# make tensorboard logs

print(FILENAME)
TENSORBOARD_LOGS_PATH=os.path.join(LOG_DIR,'_'+FILENAME).replace(" ", "_")


if not  os.path.exists(TENSORBOARD_LOGS_PATH):

    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

    # Convert the resulting series of lists to a NumPy array
    numpy_array = df.to_numpy()

    # resize images 
    images = convert_images_for_tensorboard(image_paths)

    # for tensorboard
    writer = SummaryWriter(TENSORBOARD_LOGS_PATH)
    writer.add_embedding(numpy_array, label_img=images)

else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

print(f'{FILENAME}  number of images:{len(image_paths)}')

MVTEC_LOCO_embeddings_df
MVTEC_LOCO_embeddings_df  number of images:3651


### run tensorboard on port forward to browser

In [8]:
!tensorboard  --logdir $TENSORBOARD_LOGS_PATH

2023-10-20 13:07:39.932508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-20 13:07:39.932545: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-20 13:07:39.932588: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 13:07:39.944385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 13:07:41.337600: W tensorflow/compiler/